In [ ]:
!pip install textattack[tensorflow]

     |████████████████████████████████| 373 kB 4.3 MB/s 
     |████████████████████████████████| 298 kB 41.5 MB/s 
     |████████████████████████████████| 322 kB 50.0 MB/s 
     |████████████████████████████████| 69 kB 5.5 MB/s 
     |████████████████████████████████| 3.3 MB 34.0 MB/s 
     |████████████████████████████████| 59 kB 5.4 MB/s 
     |████████████████████████████████| 769 kB 38.4 MB/s 
     |████████████████████████████████| 101 kB 9.4 MB/s 
     |████████████████████████████████| 454.3 MB 14 kB/s 
     |████████████████████████████████| 462 kB 37.1 MB/s 
     |████████████████████████████████| 4.9 MB 33.6 MB/s 
     |████████████████████████████████| 124 kB 42.9 MB/s 
     |████████████████████████████████| 4.0 MB 37.6 MB/s 
     |████████████████████████████████| 1.2 MB 38.2 MB/s 
     |████████████████████████████████| 4.9 MB 36.4 MB/s 
     |████████████████████████████████| 4.4 MB 36.2 MB/s 
     |████████████████████████████████| 4.3 MB 36.9 MB/s 
     |██████████████

In [ ]:
import textattack
import torchtext
import torch
import torch
from torchtext.datasets import IMDB, AG_NEWS, YahooAnswers
from torchtext.vocab import GloVe
from torchtext.data import to_map_style_dataset
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam
import torch.nn.functional as F
from torch.nn import LSTM, GRU, Linear, Softmax, Conv2d, Dropout
from tqdm import tqdm
import nltk
from nltk.corpus import wordnet as wn
import numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_set = YahooAnswers(split='train')
test_set = YahooAnswers(split='test')
num_classes = 10
tokenizer = get_tokenizer('basic_english')
embedding = GloVe(name='6B', dim=50)

In [ ]:
def build_thesaurus(all_words):
    # Expects torch.vocab.itos dictionary to extract thesaurus
    thesaurus = {}
    syns = []
    length_thesaurus = len(all_words)

    for i in range(length_thesaurus):
        if i % 10000 == 0:
            print("At index {} of the vocabulary".format(i))
        # Extract the word
        token = all_words[i]

        # Find the synsets for the token
        synsets = wn.synsets(token)

        if len(synsets) == 0:
            thesaurus[token] = ""
        
        else:
            # Iterate through all synset
            for synset in synsets:
                lemma_names = synset.lemma_names()
                for lemma in lemma_names:
                    # Check if lemma has an underscore indicating a two word token
                    if not("_" in lemma):
                        lemma = lemma.lower()
                        if (lemma != token and lemma not in syns):
                            syns.append(lemma)
                    
            
            thesaurus[token] = syns
            syns = []
        
    return thesaurus

In [ ]:
example_thes = build_thesaurus(embedding.itos)

At index 0 of the vocabulary
At index 10000 of the vocabulary
At index 20000 of the vocabulary
At index 30000 of the vocabulary
At index 40000 of the vocabulary
At index 50000 of the vocabulary
At index 60000 of the vocabulary
At index 70000 of the vocabulary
At index 80000 of the vocabulary
At index 90000 of the vocabulary
At index 100000 of the vocabulary
At index 110000 of the vocabulary
At index 120000 of the vocabulary
At index 130000 of the vocabulary
At index 140000 of the vocabulary
At index 150000 of the vocabulary
At index 160000 of the vocabulary
At index 170000 of the vocabulary
At index 180000 of the vocabulary
At index 190000 of the vocabulary
At index 200000 of the vocabulary
At index 210000 of the vocabulary
At index 220000 of the vocabulary
At index 230000 of the vocabulary
At index 240000 of the vocabulary
At index 250000 of the vocabulary
At index 260000 of the vocabulary
At index 270000 of the vocabulary
At index 280000 of the vocabulary
At index 290000 of the vocab

In [ ]:
def mask_replace_with_syns_add_noise(sentence, thesaurus, embedding, mask_probability=0.1, synonym_probability=0.25, pos_noise=0.1):
    tokens_to_ret = []
    for word in sentence:
        mask_flag = np.random.choice([0, 1], replace=False, p=[1-mask_probability, mask_probability])
        # Not masked
        if mask_flag == 0:
            syn_flag = np.random.choice([0, 1], replace=False, p=[1-synonym_probability, synonym_probability])
            # Not masked & replaced with synonym
            if syn_flag == 1:
                # Check if word exists in thesaurus
                synonyms = thesaurus.get(word)
                if synonyms != None and len(synonyms) != 0:
                    # randomly sample a synonym word
                    indx = np.random.randint(low=0, high=len(synonyms))
                    tokens_to_ret.append(synonyms[indx])
                # Synonym doesn't exist
                else:
                    tokens_to_ret.append(word)
            # Not masked & not replaced with synonym
            else:
                tokens_to_ret.append(word)
        # Masked
        else:
            tokens_to_ret.append("")
        
    
    # We have masked and replaced with synonyms randomly, now obtain embeddings
    embed = embedding.get_vecs_by_tokens(tokens_to_ret)
    '''pos_encoding = np.zeros(embed.shape)
    # Positional encoding introduced in Vaswani et. al.
    for i in range(embed.shape[0]):
        if i%2 == 0:
            pos_param = pos_noise*np.sin(i / (10000 ** ((2*(i//2) / embed.shape[1]))))
        else:
            pos_param = pos_noise*np.cos(i / (10000 ** ((2*(i//2) / embed.shape[1]))))'''
    return embed

In [ ]:
from torch.utils.data import Dataset

class ClassificationDataset(Dataset):
    def __init__(self, dataset, num_classes, tokenizer):
        self.num_classes = num_classes
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return self.dataset.__len__()

    def __getitem__(self, idx):
        label, text = self.dataset.__getitem__(idx)
        return int(label) - 1, self.tokenizer(text)

In [ ]:
def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _tokens) in batch:
        label_list.append(_label)
        embed = embedding.get_vecs_by_tokens(_tokens)
        text_list.append(embed)
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = pad_sequence(text_list, batch_first=True)
    return label_list.to(device), text_list.to(device)

def collate_defence_batch(batch):
    label_list, text_list = [], []
    for (_label, _tokens) in batch:
        label_list.append(_label)
        #embed = embedding.get_vecs_by_tokens(_tokens)
        embed = mask_replace_with_syns_add_noise(_tokens, example_thes, embedding)
        text_list.append(embed)
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = pad_sequence(text_list, batch_first=True)
    return label_list.to(device), text_list.to(device)

In [ ]:
train_set = to_map_style_dataset(train_set)
test_set = to_map_style_dataset(test_set)
train_set = ClassificationDataset(train_set, num_classes, tokenizer)
test_set = ClassificationDataset(test_set, num_classes, tokenizer)
BATCH_SIZE = 64
SHUFFLE = True
NUM_EPOCHS = 3
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE,
                          collate_fn=collate_defence_batch, shuffle=SHUFFLE)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE,
                         collate_fn=collate_batch, shuffle=SHUFFLE)


In [ ]:
def evaluate(model, data_loader, loss=CrossEntropyLoss()):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (labels, text) in enumerate(data_loader):
            predicted_label = model(text)
            loss_ = loss(predicted_label, labels)
            total_acc += (predicted_label.argmax(1) == labels).sum().item()
            total_count += labels.size(0)
    return total_acc / total_count


def train(model, optimizer, train_loader, loss=CrossEntropyLoss(), log_interval=50):
    model.train()
    total_acc, total_count = 0, 0
    pbar = tqdm(total=len(train_loader),
                desc=f'Epoch [{epoch + 1}/{NUM_EPOCHS}]')
    for idx, (labels, text) in enumerate(train_loader):
        output = model(text)
        loss_ = loss(output, labels)
        optimizer.zero_grad()
        loss_.backward()
        optimizer.step()
        total_acc += (output.argmax(1) == labels).sum().item()
        total_count += labels.size(0)
        pbar.update()
        if idx % log_interval == 0 and idx > 0:
            pbar.set_postfix(loss=loss_, accuracy=total_acc / total_count)
            total_acc, total_count = 0, 0

    pbar.close()

In [ ]:
class CNNClassifier(torch.nn.Module):
    def __init__(self, num_classes, in_channels, out_channels, kernel_heights, pad=0, stri=1, embed_dim=50, drop=0.2):
        super().__init__()
        self.conv1 = Conv2d(in_channels, out_channels[0], kernel_size=(kernel_heights[0], embed_dim), stride=stri, padding=pad)
        self.conv2 = Conv2d(in_channels, out_channels[1], kernel_size=(kernel_heights[1], embed_dim), stride=stri, padding=pad)
        self.conv3 = Conv2d(in_channels, out_channels[2], kernel_size=(kernel_heights[2], embed_dim), stride=stri, padding=pad)
        self.drop = Dropout(drop)
        self.fc = Linear(sum(out_channels), num_classes)
        self.soft = Softmax(dim=1)
    
    def _conv_n_maxpool_1d(self, input, conv_layer):

        conved = conv_layer(input) # conved.size() = (batch_size, out_channels[0], dim, 1)
        reld = F.relu(conved.squeeze(3)) # reld.size() = (batch_size, out_channels[0], dim)
        max_out = F.max_pool1d(reld, reld.size()[2]).squeeze(2) # maxpool_out.size() = (batch_size, out_channels[0])

        return max_out

    def forward(self, x):
        # x.size() = (batch_size, num_seq, embed_dim)
        x = x.unsqueeze(1) # x.size() = (batch_size, 1, num_seq, embed_dim)

        out_1 = self._conv_n_maxpool_1d(x, self.conv1)
        out_2 = self._conv_n_maxpool_1d(x, self.conv2)
        out_3 = self._conv_n_maxpool_1d(x, self.conv3)

        cat_out = torch.cat((out_1, out_2, out_3), dim=1)

        drop = self.drop(cat_out)
        fc_out = self.fc(drop)
        out = self.soft(fc_out)

        return out

In [ ]:
model = CNNClassifier(num_classes, in_channels=1, out_channels=(3,5,7), kernel_heights=(2,3,4)).to(device)
optim = Adam(model.parameters())

for epoch in range(3):
    train(model, optim, train_loader)

Epoch [1/3]:   2%|▏         | 525/21875 [09:00<5:53:23,  1.01it/s, accuracy=0.328, loss=tensor(2.1149, grad_fn=<NllLossBackward0>)]

KeyboardInterrupt: ignored

In [ ]:
test_accuracy = evaluate(model, test_loader)
print(f'Test accuracy: {test_accuracy}')

Test accuracy: 0.8842105263157894


In [ ]:
class CustomPyTorchModelWrapper(textattack.models.wrappers.model_wrapper.ModelWrapper):
    def __init__(self, model, outdim, vocab=torchtext.vocab.GloVe("6B", dim=50), tokenizer=torchtext.data.utils.get_tokenizer("basic_english")):
        self.model = model
        self.tokenizer = tokenizer
        self.outdim = outdim
        self.vocab = vocab
    
    def __call__(self, text_input_list):
        preds = torch.zeros(size=(len(text_input_list),self.outdim))
        for i, review in enumerate(text_input_list):
            tokens = self.tokenizer(review)
            input = self.vocab.get_vecs_by_tokens(tokens)
            with torch.no_grad():
                prediction = self.model(torch.unsqueeze(input,dim=0).to(device))
                preds[i] = prediction

        return preds

In [ ]:
model_wrapper = CustomPyTorchModelWrapper(model, outdim=num_classes)

dataset = textattack.datasets.HuggingFaceDataset("ag_news", split="test")
#attack = textattack.attack_recipes.faster_genetic_algorithm_jia_2019.FasterGeneticAlgorithmJia2019.build(model_wrapper)
attack = textattack.attack_recipes.pwws_ren_2019.PWWSRen2019.build(model_wrapper)
attack_args = textattack.AttackArgs(num_examples=1000)
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)
textattack: Loading datasets dataset ag_news, split test.
textattack: Unknown if model of class <class '__main__.CNNClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/1000 [00:00<04:09,  4.01it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Business (100%)]] --> [[World (97%)]]

Fears for T N pension after talks [[Unions]] representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent [[firm]] Federal Mogul.

Fears for T N pension after talks [[conjugation]] representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent [[firmly]] Federal Mogul.




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   0%|          | 2/1000 [00:00<05:48,  2.86it/s]

...

--------------------------------------------- Result 998 ---------------------------------------------
[[Sports (100%)]] --> [[[FAILED]]]

Roddick blisters junior champ By the time his match with Andy Roddick was over, Jenkins had felt the full fury of Roddick #39;s jet blast. Roddick had nailed a 152-mph serve at him, the fastest serve in Open history and one 




[Succeeded / Failed / Skipped / Total] 671 / 198 / 130 / 999: 100%|█████████▉| 999/1000 [05:54<00:00,  2.82it/s]

--------------------------------------------- Result 999 ---------------------------------------------
[[Sci/tech (100%)]] --> [[[FAILED]]]

AMD dual-core demo pips Intel, IBM AMD has demonstrated the company #39;s first dual-core microprocessors. Dual-core processors offer improved performance over single-core chips, especially in multithreaded applications.




[Succeeded / Failed / Skipped / Total] 671 / 199 / 130 / 1000: 100%|██████████| 1000/1000 [05:54<00:00,  2.82it/s]

--------------------------------------------- Result 1000 ---------------------------------------------
[[World (100%)]] --> [[[FAILED]]]

Gunmen ambush Chalabi #39;s convoy, wound 2 BAGHDAD - Gunmen ambushed the convoy of former Iraqi governing council president Ahmed Chalabi on Wednesday, wounding two of his bodyguards, aides said.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 671    |
| Number of failed attacks:     | 199    |
| Number of skipped attacks:    | 130    |
| Original accuracy:            | 87.0%  |
| Accuracy under attack:        | 19.9%  |
| Attack success rate:          | 77.13% |
| Average perturbed word %:     | 12.32% |
| Average num. words per input: | 39.54  |
| Avg num queries:              | 323.79 |
+-------------------------------+--------+


In [ ]:
!pip install torchfile

  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5710 sha256=0b3b369d19f1b4d0c61517d6c149ad7add22f65170cc125f96dd88c53c97662b
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built torchfile


In [ ]:
idx = 1

In [ ]:
print(train_dataset.__getitem__(idx)[0]["text"])#(0)["text"])
print(train_dataset.__getitem__(idx)[1])

Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
2


In [ ]:
txt = train_dataset.__getitem__(idx)[0]["text"]
lbl = train_dataset.__getitem__(idx)[1]

In [ ]:
res = attack.attack(txt, lbl)

['Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.']
['[UNK] Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.', 'Carlyle [UNK] Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.', 'Carlyle Looks [UNK] Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial pl

In [ ]:
new_txt = res.perturbed_text(color_method="red")

In [ ]:
txt

'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.'

In [ ]:
new_txt

'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for piss well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.'

SYNONYMS

In [ ]:
embedding = GloVe(name='6B', dim=50)

.vector_cache/glove.6B.zip: 862MB [02:42, 5.31MB/s]                           
100%|█████████▉| 399999/400000 [00:11<00:00, 35875.02it/s]


In [ ]:
embedding.itos[1:100]

[',',
 '.',
 'of',
 'to',
 'and',
 'in',
 'a',
 '"',
 "'s",
 'for',
 '-',
 'that',
 'on',
 'is',
 'was',
 'said',
 'with',
 'he',
 'as',
 'it',
 'by',
 'at',
 '(',
 ')',
 'from',
 'his',
 "''",
 '``',
 'an',
 'be',
 'has',
 'are',
 'have',
 'but',
 'were',
 'not',
 'this',
 'who',
 'they',
 'had',
 'i',
 'which',
 'will',
 'their',
 ':',
 'or',
 'its',
 'one',
 'after',
 'new',
 'been',
 'also',
 'we',
 'would',
 'two',
 'more',
 "'",
 'first',
 'about',
 'up',
 'when',
 'year',
 'there',
 'all',
 '--',
 'out',
 'she',
 'other',
 'people',
 "n't",
 'her',
 'percent',
 'than',
 'over',
 'into',
 'last',
 'some',
 'government',
 'time',
 '$',
 'you',
 'years',
 'if',
 'no',
 'world',
 'can',
 'three',
 'do',
 ';',
 'president',
 'only',
 'state',
 'million',
 'could',
 'us',
 'most',
 '_',
 'against',
 'u.s.']

In [ ]:
embedding.itos[231232]

'antiemetic'

In [ ]:
len(embedding.itos)

400000

In [ ]:
nltk.download('wordnet')

[Synset('preside.v.01')]

In [ ]:
synonym = wordnet.synsets("working")
synonym

[Synset('working.n.01'),
 Synset('work.v.01'),
 Synset('work.v.02'),
 Synset('work.v.03'),
 Synset('function.v.01'),
 Synset('work.v.05'),
 Synset('exercise.v.03'),
 Synset('make.v.36'),
 Synset('work.v.08'),
 Synset('work.v.09'),
 Synset('work.v.10'),
 Synset('bring.v.03'),
 Synset('work.v.12'),
 Synset('cultivate.v.02'),
 Synset('work.v.14'),
 Synset('influence.v.01'),
 Synset('work.v.16'),
 Synset('work.v.17'),
 Synset('work.v.18'),
 Synset('work.v.19'),
 Synset('shape.v.02'),
 Synset('work.v.21'),
 Synset('knead.v.01'),
 Synset('exploit.v.01'),
 Synset('solve.v.01'),
 Synset('ferment.v.03'),
 Synset('sour.v.01'),
 Synset('work.v.27'),
 Synset('working.s.01'),
 Synset('working.s.02'),
 Synset('working.s.03'),
 Synset('running.s.06'),
 Synset('working.s.05')]

In [ ]:


synonym = wordnet.synsets("working")
synset = synonym[0]
lemma_names = synset.lemma_names()

print(synonym)
print(synset)
print(lemma_names)


[Synset('working.n.01'), Synset('work.v.01'), Synset('work.v.02'), Synset('work.v.03'), Synset('function.v.01'), Synset('work.v.05'), Synset('exercise.v.03'), Synset('make.v.36'), Synset('work.v.08'), Synset('work.v.09'), Synset('work.v.10'), Synset('bring.v.03'), Synset('work.v.12'), Synset('cultivate.v.02'), Synset('work.v.14'), Synset('influence.v.01'), Synset('work.v.16'), Synset('work.v.17'), Synset('work.v.18'), Synset('work.v.19'), Synset('shape.v.02'), Synset('work.v.21'), Synset('knead.v.01'), Synset('exploit.v.01'), Synset('solve.v.01'), Synset('ferment.v.03'), Synset('sour.v.01'), Synset('work.v.27'), Synset('working.s.01'), Synset('working.s.02'), Synset('working.s.03'), Synset('running.s.06'), Synset('working.s.05')]
Synset('working.n.01')
['working', 'workings']


In [ ]:
example_sentence = "the match today against the golden state warriors was great we won by ten points"
tokenizer = get_tokenizer('basic_english')
tokenized_sentence = tokenizer(example_sentence)

In [ ]:
tokenized_sentence

['the',
 'match',
 'today',
 'against',
 'the',
 'golden',
 'state',
 'warriors',
 'was',
 'great',
 'we',
 'won',
 'by',
 'ten',
 'points']

In [ ]:
example_thes.keys()

In [ ]:
synonym_replaced = []
for word in tokenized_sentence:
    # Sample the probability if this word is to be replaced by the synonym
    flag = np.random.randint(low=0, high=2)
    # Check if word exists in corpus
    # to do

    if flag == 1:
        synonyms = example_thes[word]
        if len(synonyms) != 0:
            # randomly sample a synonym word
            indx = np.random.randint(low=0, high=len(synonyms))
            synonym_replaced.append(synonyms[indx])
        else:
            synonym_replaced.append(word)
    else:
        synonym_replaced.append(word)

synonym_replaced

['the',
 'match',
 'today',
 'against',
 'the',
 'gilt',
 'state',
 'warriors',
 'comprise',
 'great',
 'we',
 'acquire',
 'past',
 'decade',
 'points']

In [ ]:
def replace_tokens_with_wordnet_syns(sentence_to_switch, thesaurus, probability=0.25):
    synonym_replaced = []
    for word in sentence_to_switch:
        # Sample the probability if this word is to be replaced by the synonym
        flag = np.random.choice([0,1], replace=False, p=[1-probability, probability])
        # Check if word exists in corpus
        # to do
        if flag == 1:
            synonyms = thesaurus[word]
            if len(synonyms) != 0:
                # randomly sample a synonym word
                indx = np.random.randint(low=0, high=len(synonyms))
                synonym_replaced.append(synonyms[indx])
            else:
                synonym_replaced.append(word)
        else:
            synonym_replaced.append(word)
    return synonym_replaced

In [ ]:
def mask_random_tokens(sentence_to_mask, probability=0.1):
    masked = []
    for word in sentence_to_mask:
        flag = np.random.choice([0,1], replace=False, p=[1-probability, probability])

        if flag == 0:
            masked.append(word)
        
        else:
            masked.append("")
    
    return masked

In [ ]:
mask_random_tokens(tokenized_sentence)

['',
 'match',
 'today',
 'against',
 'the',
 'golden',
 'state',
 '',
 'was',
 'great',
 'we',
 'won',
 'by',
 'ten',
 '']

In [ ]:
replace_tokens_with_wordnet_syns(tokenized_sentence, example_thes)

['the',
 'match',
 'now',
 'against',
 'the',
 'golden',
 'state',
 'warriors',
 'was',
 'great',
 'we',
 'won',
 'past',
 'ten',
 'points']